# Labware Manipulation/Site v2.0

### What we have now

In [ ]:
# The bad (1) - PrepareForInput (and output)

from typing import NamedTuple
HandoverPosition = NamedTuple("HandoverPosition", Position=str, Index=int)


async def AvailableHandoverPositions(self) -> list[HandoverPosition]:
    return []

async def NumberOfInternalPositions(self) -> int:
    return 1

async def PrepareForInput(
    HandoverPosition: HandoverPosition,
    InternalPosition: int,
    LabwareType: str,
    LabwareUniqueID: str,
) -> None:
    pass

async def PrepareForOutput(
    HandoverPosition: HandoverPosition,
    InternalPosition: int,
) -> None:
    pass


# HandoverPosition - why do we need booth Position and Index? Why booth, why not just one? Arbitrary much?
#                   - And how do I map this position to the actual robot? Need an external lookup table anyway.
# InternalPosition - Why just an index here? (see HandoverPosition) How do I know which are free?

# LabwareType - sure I can tell you what I'm gonna pass, so some logic can be applied.... but I'm also 
#               telling you exactly where to put it. So what is the point? Also, not used for retrieval.
# LabwareUniqueID - You were given a position? Why do you need a unique ID? It is not used for retrieval, why is it needed?


# No return value ???
# Wouldn't it be nice if a token was returned, so we could use that to refer to this call later on?
# - PrepareForInput -> GetLabware.
# Or return the internal position where you plan to place it...

In [ ]:
# The bad (2) - Put/Get/Delivered/Removed

async def GetLabware(
        HandoverPosition: HandoverPosition,
        IntermediateActions: list[str],
    ) -> None:
        pass

# HandoverPosition - see comment above. Replace with token here?
# IntermediateActions - Why? Why not just call a series of sila commands directly?

### The Fix

In [ ]:
# The fix (1) - PrepareForInput (and output)

HandoverposUUID = str
InternalPosUUID = str
LabwareUUID = str
TransactionToken = str

# Change to return UUIDs
async def AvailableHandoverPositions(self) -> list[HandoverposUUID]:
    return []


# Change to return a InternalPosition to Labware map
# I know SiLA XML does not support null values... wish for the future
async def GetContent(self) -> dict[InternalPosUUID, LabwareUUID | None]:
    return dict()


# Remove: InternalPosition, LabwareUniqueID, LabwareType
# Changed to return a token
async def PrepareForInput(
        HandOverPosition: HandoverposUUID,
    ) -> TransactionToken:
    return ""

# Remove: InternalPosition
# Added: LabwareUUID
# Changed to return a token
async def PrepareForOutput(
        HandOverPosition: HandoverposUUID,
        LabwareUniqueID: LabwareUUID,
    ) -> TransactionToken:
    return ""


In [ ]:
# The fix (2) - Put/Get/Delivered/Removed

# Removed IntermediateActions, HandoverPosition
# Added: TransactionToken
async def PutLabware(
        Token: TransactionToken,
    ) -> None:
        pass

# Added: Returns InternalPosUUID - where the labware is placed in the end
async def LabwareDelivered(
        Token: TransactionToken,
        LabwareUniqueID: LabwareUUID,
    ) -> InternalPosUUID:
        return ""


### What it would look like

In [ ]:
from unittest.mock import MagicMock # Stops IDE from complaining about missing methods

# SiLA Devices
SRC_DEVICE = MagicMock()
TGT_DEVICE = MagicMock()

# Data
plate = {"unique_id": "1234", "process_to_run": "PCR-day1"}

# 0. Get handover position (UUID)
handover = set(await SRC_DEVICE.AvailableHandoverPositions()) & set(await TGT_DEVICE.AvailableHandoverPositions()).pop()

# 1. Prepare for input/output
src_token = await SRC_DEVICE.PrepareForOutput(handover, plate["unique_id"])
tgt_token = await TGT_DEVICE.PrepareForInput(handover)

# 2. Let's assume the source is active
await SRC_DEVICE.PutLabware(src_token)
end_pos = await TGT_DEVICE.LabwareDelivered(tgt_token, plate["unique_id"])
# end_pos should be the internal position where the plate was placed

# 3. Extra
# Since the target device now know what is loaded, we can do this
await TGT_DEVICE.RunProcess(plate["process_to_run"])
# And it should.... "hand-wave".... know what wells are to be processed, and where/how to document the results. 
